# Running several optimizations and keeping the best one with MultipleLaunchesPlugin

In this notebook, we present the `MultipleLaunchesAnalyzer` plugin. Placed before a VQE plugin, it will run the job for several iterations with different random parameters at each run. It will then select the lowest observable value it has found among these runs. 

More specifically, placing the `MultipleLaunchesAnalyzer` plugin set with $n_{\mathrm{runs}}=10$ before an `Optimizer` instantiated without any starting point will allow the ansatz to be optimized $n_{\mathrm{runs}}$ times, each time with a different set of random parameters. The output value will be the lowest found value obtained through the iteration process.

This is particularly useful when the ansatz parameters values cannot be guessed before a VQE.

**Note**: if a job in the batch is not variational, it will be evaluated 10 times!

## Example : Solving an embedded model using `MultipleLaunchesAnalyzer`

We will solve an embedded model using the `MultipleLaunchesAnalyzer`. Let us say we are unsure about which ansatz would be best suited for this task. We select two ansatz :
* the 8-parameters hardware-efficient circuit proposed by (Keen et al., 2019) (https://doi.org/10.48550/arXiv.1910.09512)
* the physically-inspired circuit (LDCA) with 34 parameters proposed by P. Dallaire-Demers et al. (2019) (https://doi.org/10.48550/arXiv.1801.01053>)

We will start by defining the Hamiltonian to solve, define the two ansatze, and then compute the observable value for the two ansatze (using `Batch`) and for different set of random parameters (using `MultipleLaunchesAnalyzer`).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from qat.qpus import get_default_qpu
from qat.core import Batch, Job

from qat.fermion.transforms import transform_to_jw_basis
from qat.fermion.hamiltonians import make_embedded_model
from qat.fermion.circuits import make_shallow_circ, make_ldca_circ

from qat.plugins import ScipyMinimizePlugin, MultipleLaunchesAnalyzer

We start by defining the Hamiltonian we'd like to find the ground state of with VQE. 

In [ ]:
# We define the Hamiltonian
U = 1
mu = U / 2
D = 0.4 * np.eye(2)
lambda_c = -0.04 * np.eye(2)
hamiltonian = make_embedded_model(U, mu, D, lambda_c, grouping="spins")

# The ordering of the orbitals (here, 'spins') is crucial for the HEA circuit

We now define two ansatze we want to use for the observable evaluation.

In [ ]:
# Defining the first job
circ1 = make_shallow_circ()
job1 = circ1.to_job(job_type="OBS", observable=transform_to_jw_basis(hamiltonian))

# Defining the second job
circ2 = make_ldca_circ(4, 1)
job2 = circ2.to_job(job_type="OBS", observable=transform_to_jw_basis(hamiltonian))

We define a `Batch` containing these two variational jobs, and define the stack to submit the `Batch` to.

In [ ]:
# Defining the batch
batch = Batch(jobs=[job1, job2])

# Build the stack and submit the job to the QPU
qpu = get_default_qpu()
multiple_launches_analyzer = MultipleLaunchesAnalyzer(n_runs=10, verbose=True)
scipy_optimizer = ScipyMinimizePlugin(method="COBYLA", tol=1e-3, options={"maxiter": 200})
stack = multiple_launches_analyzer | scipy_optimizer | qpu

res = stack.submit(batch)

In [ ]:
# Select the lowest obtained energy
energy = min((result.value for _, result in enumerate(res)))
print(f"We obtained an energy of {energy}.")

In [ ]:
# Exact diagonalization of the Hamiltonian for future reference
eigvals_cl = np.linalg.eigvalsh(hamiltonian.get_matrix())
E0 = min(eigvals_cl)
print(f"Exact ground state = {E0}")

In [ ]:
print(
    "Variance of results obtained by optimizing the small hardware-efficient circuit:", res[0].meta_data["optimal_values_variance"]
)
print("Variance of results obtained by optimizing the LDCA circuit:", res[1].meta_data["optimal_values_variance"])

As expected, the variance of the results is a lot greater for the small circuit, which is the sign of an increased sensitivity to the initial parameters. 